In [54]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
%matplotlib inline

import matplotlib.pyplot as plt

In [55]:
base_dataset_dir = "E:\Major Project 2022\Dataset"
csv_path = base_dataset_dir+"\\"+"64by64_Dataset_RBG.csv"

In [56]:
image_df = pd.read_csv(csv_path)

In [57]:
image_df.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,12279,12280,12281,12282,12283,12284,12285,12286,12287,Label
5614,86,75,80,98,87,90,100,89,95,97,...,200,167,180,196,163,180,191,161,177,2
9435,9,7,10,10,8,11,10,8,11,12,...,43,33,28,29,21,19,15,9,10,4
11954,231,165,151,226,147,125,223,148,121,196,...,223,130,106,225,143,122,224,133,105,5
15540,241,207,191,241,209,194,241,213,203,240,...,213,162,159,201,141,131,196,132,116,7
1705,74,63,55,67,56,50,35,31,30,2,...,0,0,0,0,0,0,0,0,0,1


In [58]:
X = image_df.drop("Label", axis=1).values
label = image_df["Label"].values

In [59]:
X.shape, label.shape

((16431, 12288), (16431,))


Scaling and Split Data into Train, Validation and Test set


In [7]:
X_mean = np.mean(X)
X_std = np.std(X)

X = (X - X_mean)/X_std

In [8]:
X_train_orig, X_test, y_train_orig, y_test = train_test_split(X, label, test_size=0.1,random_state=0)

X_train, X_val, y_train, y_val = train_test_split(X_train_orig, y_train_orig, test_size=0.2, random_state=1)

X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((11829, 12288), (2958, 12288), (1644, 12288), (11829,), (2958,), (1644,))


Reshape the Data to Input in CNN


In [9]:
X_train = X_train.reshape(X_train.shape[0], *(64, 64, 3))
X_val = X_val.reshape(X_val.shape[0], *(64, 64, 3))
X_test = X_test.reshape(X_test.shape[0], *(64, 64, 3))

X_train.shape, X_val.shape, X_test.shape

((11829, 64, 64, 3), (2958, 64, 64, 3), (1644, 64, 64, 3))

In [10]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

y_train.shape, y_val.shape, y_test.shape

((11829, 8), (2958, 8), (1644, 8))

# CNN Model using Tensorflow

In [42]:
# Our input feature map is 64x64x3: 64x64 for the image pixels, and 3 for
# the three color channels: R, G, and B
img_input = layers.Input(shape=(64, 64, 3))

# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(16, 3, activation='relu', padding='same')(img_input)
x = layers.MaxPooling2D(2)(x)

# Second convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2)(x)

# Third convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Convolution2D(64, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2)(x)

# Flatten feature map to a 1-dim tensor
x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Create output layer with a single node and sigmoid activation
output = layers.Dense(7, activation='softmax')(x)

# Configure and compile the model
model = Model(img_input, output)

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 64, 64, 16)        448       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 16, 16, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 16, 16, 64)        1849


Define Data Generator for Data Augmentation and Learning Rate Adaptive Reduction to Prevent Overfitting


In [45]:
#X_train_np = np.array(X_train)
#model.fit(X_train_np,output_labels,epochs=30)

In [46]:
X_train_np

array([[[[ 1.05867141,  1.17758947,  1.61928511],
         [ 0.37913965,  0.71890553,  1.43241388],
         [-1.50656098, -1.31968975, -0.86100581],
         ...,
         [-1.52354927, -1.48957269, -1.1667951 ],
         [-1.76138539, -1.93126833, -1.76138539],
         [-1.7274088 , -1.84632686, -1.65945563]],

        [[ 0.99071823,  1.19457776,  1.67024999],
         [-0.21545064,  0.19226842,  0.92276506],
         [-1.77837368, -1.77837368, -1.54053757],
         ...,
         [-1.47258439, -1.33667804, -0.91197069],
         [-1.76138539, -1.93126833, -1.79536198],
         [-1.71042051, -1.84632686, -1.64246733]],

        [[ 0.78685871,  1.0756597 ,  1.6532617 ],
         [-0.94594728, -0.6061814 ,  0.00539718],
         [-1.82933857, -1.91428004, -1.77837368],
         ...,
         [-1.48957269, -1.30270145, -0.82702922],
         [-1.7443971 , -1.91428004, -1.77837368],
         [-1.69343221, -1.84632686, -1.64246733]],

        ...,

        [[ 1.14361288,  1.21156606,  1

KeyError: 'Label'

ValueError: Data cardinality is ambiguous:
  x sizes: 11829
  y sizes: 16431
Make sure all arrays contain the same number of samples.

In [35]:
train_datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,
                             shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

train_datagen.fit(X_train)

val_datagen = ImageDataGenerator()
val_datagen.fit(X_val)

In [32]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [38]:
batch_size = 64
epochs = 30
train_generator = train_datagen.flow(X_train,y_train, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val)

history = model.fit(train_generator ,epochs = 30 , validation_data = val_generator,verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),callbacks=[learning_rate_reduction])

Epoch 1/30


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "C:\Users\DELL\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\DELL\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\DELL\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\DELL\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\DELL\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\DELL\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\DELL\anaconda3\lib\asyncio\base_events.py", line 596, in run_fore

In [28]:
loss_test, acc_test = model.evaluate(X_test, y_test, verbose=1)
loss_val, acc_val = model.evaluate(X_val, y_val, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))
print("Test: accuracy = %f  ;  loss = %f" % (acc_test, loss_test))
model.save("model.h5")

ValueError: in user code:

    File "C:\Users\DELL\anaconda3\lib\site-packages\keras\engine\training.py", line 1525, in test_function  *
        return step_function(self, iterator)
    File "C:\Users\DELL\anaconda3\lib\site-packages\keras\engine\training.py", line 1514, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\DELL\anaconda3\lib\site-packages\keras\engine\training.py", line 1507, in run_step  **
        outputs = model.test_step(data)
    File "C:\Users\DELL\anaconda3\lib\site-packages\keras\engine\training.py", line 1473, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\DELL\anaconda3\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "C:\Users\DELL\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\Users\DELL\ana